In [1]:
from dlt.sources.helpers.rest_client import RESTClient
from dlt.sources.helpers.rest_client.auth import BearerTokenAuth
from dlt.sources.helpers.rest_client.paginators import HeaderLinkPaginator , JSONLinkPaginator

BASE_URL = "https://pokeapi.co/api/v2/pokemon?limit=10"

client = RESTClient(
    base_url= BASE_URL,
    paginator=JSONLinkPaginator(next_url_path="next"),
    data_selector="results",
)

count = 0
max_count = 10

for page in client.paginate():
    for pokemon in page:
        print(pokemon["name"])
        count += 1
        if count > max_count:
            break 
    if count >= max_count:
        break 
    print(page)

bulbasaur
ivysaur
venusaur
charmander
charmeleon
charizard
squirtle
wartortle
blastoise
caterpie


In [2]:
#DLT Resources: 
import dlt 
from dlt.sources.helpers.rest_client import RESTClient
from dlt.sources.helpers.rest_client.auth import BearerTokenAuth
from dlt.sources.helpers.rest_client.paginators import HeaderLinkPaginator , JSONLinkPaginator

BASE_URL = "https://pokeapi.co/api/v2/pokemon?limit=10"

@dlt.resource()
def paginated_gatter():
    client = RESTClient(
        base_url= BASE_URL,
        paginator=JSONLinkPaginator(next_url_path="next"),
        data_selector="results",
    )

    count = 0
    max_count = 10

    for page in client.paginate():
        for pokemon in page:
            print(pokemon["name"])
            count += 1
            if count > max_count:
                break 
        if count >= max_count:
            break 
        yield page 
for record in paginated_gatter():
    print(record)
    

bulbasaur
ivysaur
venusaur
charmander
charmeleon
charizard
squirtle
wartortle
blastoise
caterpie


In [3]:
#dlt config and secrets: 
"""import os
import dlt
from dlt.sources.helpers.rest_client import RESTClient
from dlt.sources.helpers.rest_client.auth import BearerTokenAuth
from dlt.sources.helpers.rest_client.paginators import HeaderLinkPaginator
from google.colab import userdata

os.environ["ACCESS_TOKEN"] = userdata.get('ACCESS_TOKEN') # <--- set ENV in the special format: uppercase secret name
events_data = []


@dlt.resource(name="events")
def paginated_getter(
    access_token=dlt.secrets.value  # <--- set the secret variable "access_token" here
):
    client = RESTClient(
        base_url="https://api.github.com/repos/DataTalksClub/data-engineering-zoomcamp/",
        auth=BearerTokenAuth(token=access_token), # <--- use the variable "access_token" as usual
        paginator=HeaderLinkPaginator(links_next_key="next"),
    )

    for page in client.paginate("events"):
        yield page


for page_data in paginated_getter():
    events_data.append(page_data)
    print(page_data)"""

'import os\nimport dlt\nfrom dlt.sources.helpers.rest_client import RESTClient\nfrom dlt.sources.helpers.rest_client.auth import BearerTokenAuth\nfrom dlt.sources.helpers.rest_client.paginators import HeaderLinkPaginator\nfrom google.colab import userdata\n\nos.environ["ACCESS_TOKEN"] = userdata.get(\'ACCESS_TOKEN\') # <--- set ENV in the special format: uppercase secret name\nevents_data = []\n\n\n@dlt.resource(name="events")\ndef paginated_getter(\n    access_token=dlt.secrets.value  # <--- set the secret variable "access_token" here\n):\n    client = RESTClient(\n        base_url="https://api.github.com/repos/DataTalksClub/data-engineering-zoomcamp/",\n        auth=BearerTokenAuth(token=access_token), # <--- use the variable "access_token" as usual\n        paginator=HeaderLinkPaginator(links_next_key="next"),\n    )\n\n    for page in client.paginate("events"):\n        yield page\n\n\nfor page_data in paginated_getter():\n    events_data.append(page_data)\n    print(page_data)'

In [ ]:
import json
import dlt
import duckdb

# Load JSON from local file
with open("trainers.json") as f:
    data = json.load(f)  # reads the entire JSON array

# Define DLT resource
@dlt.resource(name="trainers", write_disposition="replace")
def load_trainers():
    for row in data:
        yield row

# Define DLT pipeline
pipeline = dlt.pipeline(
    pipeline_name="pokemon_pipeline",
    destination="duckdb",  # or "bigquery", "postgres", etc.
    dataset_name="pokemon"
)

# Run the pipeline
info = pipeline.run(load_trainers)
print(info)


print(pipeline.dataset().schema.data_table_names())


Pipeline pokemon_pipeline load step completed in 1.33 seconds
1 load package(s) were loaded to destination duckdb and into dataset pokemon
The duckdb destination used duckdb:///h:\Data Engineering\Projects\DE Projects\ETL_DLT_pipeline\pokemon_pipeline.duckdb location to store data
Load package 1756006310.096663 is LOADED and contains no failed jobs
['trainers', 'trainers__pokemon', 'trainers__pokemon__type']


In [8]:
#Shows the tables in the db
import duckdb
import numpy as np 
con = duckdb.connect("pokemon_pipeline.duckdb")

# Query the actual table name
result = con.execute("SELECT * FROM pokemon.trainers").fetchdf()
print(result)


   id         name  age location__city location__region       _dlt_load_id  \
0   1  Ash Ketchum   10    Pallet Town            Kanto  1756006310.096663   
1   2        Misty   12  Cerulean City            Kanto  1756006310.096663   

          _dlt_id  
0  6TtNoXJuS7kqaQ  
1  aX3y4rr0CZpCfA  


In [ ]:
#Installs numpy and pandas in the same kernel 
import sys
!"{sys.executable}" -m pip install numpy pandas


   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   --------- ------------------------------ 2.9/12.8 MB 15.2 MB/s eta 0:00:01
   --------------------- ------------------ 6.8/12.8 MB 17.5 MB/s eta 0:00:01
   ---------------------------------------  12.6/12.8 MB 21.3 MB/s eta 0:00:01
   ---------------------------------------- 12.8/12.8 MB 19.1 MB/s  0:00:00
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---------------------- ----------------- 6.3/11.0 MB 29.6 MB/s eta 0:00:01
   ---------------------------------------  10.7/11.0 MB 29.1 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 25.4 MB/s  0:00:00

   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   --------------------------------

In [ ]:
#Schema of the table 
schema = con.execute("DESCRIBE pokemon.trainers").fetchdf()
print(schema)


        column_name column_type null   key default extra
0                id      BIGINT  YES  None    None  None
1              name     VARCHAR  YES  None    None  None
2               age      BIGINT  YES  None    None  None
3    location__city     VARCHAR  YES  None    None  None
4  location__region     VARCHAR  YES  None    None  None
5      _dlt_load_id     VARCHAR   NO  None    None  None
6           _dlt_id     VARCHAR   NO  None    None  None


In [28]:
#Flatten the NESTED JSON:
#Sample Nested JSON data:
import pandas as pd

# Example nested JSON with varying numbers of Pokémon and possibly missing fields
data = [
    {
        "name": "Ash",
        "age": 10,
        "location": {"city": "Pallet Town", "region": "Kanto"},
        "pokemon": [
            {"name": "Pikachu", "type": "Electric"},
            {"name": "Bulbasaur", "type": "Grass/Poison"}
        ]
    },
    {
        "name": "Misty",
        "age": 10,
        "location": {"city": "Cerulean City"},  # region missing
        "pokemon": [
            {"name": "Staryu", "type": "Water"},
            {"name": "Psyduck"}  # type missing
        ]
    },
    {
        "name": "Brock",
        "age": 15,
        "location": {"city": "Pewter City", "region": "Kanto"},
        "pokemon": []  # no Pokémon
    }
]

# Flatten the JSON
df = pd.json_normalize(
    data,
    record_path=['pokemon'],  # nested list to explode into rows
    meta=['name', 'age', ['location', 'city'], ['location', 'region']],
    record_prefix="pokemon_",
    errors='ignore'
)
print(df)

  pokemon_name  pokemon_type   name age  location.city location.region
0      Pikachu      Electric    Ash  10    Pallet Town           Kanto
1    Bulbasaur  Grass/Poison    Ash  10    Pallet Town           Kanto
2       Staryu         Water  Misty  10  Cerulean City             NaN
3      Psyduck           NaN  Misty  10  Cerulean City             NaN


In [31]:
#Handling missing values:
df.fillna('', inplace=True)

#Rename columns:
df = df.rename(columns={
    'name': 'pokemon_name',
    'type': 'pokemon_type',
    'name': 'trainer_name',
    'location.city' : 'city',
    'location.region': 'region'
})
print(df)

  pokemon_name  pokemon_type trainer_name age           city region
0      Pikachu      Electric          Ash  10    Pallet Town  Kanto
1    Bulbasaur  Grass/Poison          Ash  10    Pallet Town  Kanto
2       Staryu         Water        Misty  10  Cerulean City       
3      Psyduck                      Misty  10  Cerulean City       


In [ ]:
#Setting up of DATA CONTRACT: 
import dlt
from dlt.sources.helpers.rest_client import RESTClient
from dlt.sources.helpers.rest_client.paginators import JSONLinkPaginator

BASE_URL = "https://pokeapi.co/api/v2/pokemon?limit=10"

@dlt.resource(schema_contract={"tables": "evolve", "columns": "freeze", "data_type": "evolve"})
def paginated_gatter(add_new_col: bool = False, max_count: int = 10):
    client = RESTClient(
        base_url=BASE_URL,
        paginator=JSONLinkPaginator(next_url_path="next"),
        data_selector="results",
    )

    count = 0
    for page in client.paginate():
        for pokemon in page:
            if add_new_col:
                # 🚨 Add new unexpected column
                pokemon["new_col_test"] = "unexpected_value"
            yield pokemon
            count += 1
            if count >= max_count:
                return

pipeline = dlt.pipeline(
    pipeline_name="pokemon_contract",
    destination="duckdb",
    dataset_name="pokemon_dataset"
)

info = pipeline.run(paginated_gatter(add_new_col=True), table_name="pokemons", write_disposition="append")
print(info)


PipelineStepFailed: Pipeline execution failed at `step=normalize` when processing package with `load_id=1756094124.197416` with exception:

<class 'dlt.normalize.exceptions.NormalizeJobFailed'>
Job for `job_id=pokemons.6ba702ac79.typed-jsonl.gz` failed terminally in load with `load_id=1756094124.197416` with message: In schema `pokemon_contract`: In Table: `pokemons` Column: `new_col_test` . Contract on `columns` with `contract_mode=freeze` is violated. Can't add table column `new_col_test` to table `pokemons` because `columns` are frozen. Offending data item: _dlt_id: S26apUUiabzPlQ.

In [ ]:
#Loading with dlt: Using default  method 
import dlt 
from dlt.sources.helpers.rest_client import RESTClient
from dlt.sources.helpers.rest_client.auth import BearerTokenAuth
from dlt.sources.helpers.rest_client.paginators import HeaderLinkPaginator , JSONLinkPaginator

BASE_URL = "https://pokeapi.co/api/v2/pokemon?limit=10"

@dlt.resource(name = "pokemon" ,write_disposition="replace")
def paginated_gatter():
    client = RESTClient(
        base_url= BASE_URL,
        paginator=JSONLinkPaginator(next_url_path="next"),
        data_selector="results",
    )


    for page in client.paginate():
        for pokemon in page:
            yield pokemon
    

#Define pipeline:
pipeline = dlt.pipeline(
    pipeline_name="Pokemon_names", 
    destination="duckdb",
    dataset_name="pokemon_dataset"
)

#Run the pipeline:
load_info = pipeline.run(paginated_gatter())  # <-- No need to specify write_disposition here

df = pipeline.dataset().pokemon.df()
print(df.head())


         name                                   url        _dlt_load_id  \
0   bulbasaur  https://pokeapi.co/api/v2/pokemon/1/  1758695014.0233572   
1     ivysaur  https://pokeapi.co/api/v2/pokemon/2/  1758695014.0233572   
2    venusaur  https://pokeapi.co/api/v2/pokemon/3/  1758695014.0233572   
3  charmander  https://pokeapi.co/api/v2/pokemon/4/  1758695014.0233572   
4  charmeleon  https://pokeapi.co/api/v2/pokemon/5/  1758695014.0233572   

          _dlt_id  
0  jGi+orkj64HXyg  
1  QNCDxlXRZtBCFg  
2  F07XC/WHc+o1yQ  
3  EmdEtmDH7KpqLw  
4  WKN9T0AkWNll+Q  


In [8]:
#Gives full TRACE of the pipeline: 
print(load_info)
print(load_info.loads_ids) 
print(load_info.asdict())
pipeline.dataset().schema.data_table_names()  #Table created

Pipeline Pokemon_names load step completed in 0.42 seconds
1 load package(s) were loaded to destination duckdb and into dataset pokemon_dataset
The duckdb destination used duckdb:///h:\Data Engineering\Projects\DE Projects\ETL_DLT_pipeline\Pokemon_names.duckdb location to store data
Load package 1758695014.0233572 is LOADED and contains no failed jobs
['1758695014.0233572']
{'pipeline': {'pipeline_name': 'Pokemon_names'}, 'destination_type': 'dlt.destinations.duckdb', 'destination_displayable_credentials': 'duckdb:///h:\\Data Engineering\\Projects\\DE Projects\\ETL_DLT_pipeline\\Pokemon_names.duckdb', 'destination_name': 'duckdb', 'environment': None, 'staging_type': None, 'staging_name': None, 'staging_displayable_credentials': None, 'destination_fingerprint': '', 'dataset_name': 'pokemon_dataset', 'loads_ids': ['1758695014.0233572'], 'load_packages': [{'load_id': '1758695014.0233572', 'package_path': 'C:\\Users\\hrish\\.dlt\\pipelines\\Pokemon_names\\load\\loaded\\1758695014.0233572'

['pokemon']

In [3]:
!pip install mysql-connector-python


  Using cached mysql_connector_python-9.4.0-cp312-cp312-win_amd64.whl.metadata (7.7 kB)
Using cached mysql_connector_python-9.4.0-cp312-cp312-win_amd64.whl (16.4 MB)


In [18]:

import dlt
import mysql.connector

@dlt.source
def mysql_source():
    def fetch():
        conn = mysql.connector.connect(
            host="127.0.0.1",
            user="root",
            password="Password@11",
            database="school"
        )
        cursor = conn.cursor(dictionary=True)
        cursor.execute("SELECT * FROM student")  # Your MySQL table
        rows = cursor.fetchall()
        cursor.close()
        conn.close()
        for row in rows:
            yield row

    # Resource name becomes the DuckDB table name
    return dlt.resource(fetch(), name="school")


In [20]:

#Define pipeline:
pipeline = dlt.pipeline(
    pipeline_name="MySql_pipelines", 
    destination="duckdb", 
    dev_mode=True,
)

# Collect the data from the MySQL source
data = mysql_source()
info = pipeline.run(data)
print(info)

2025-09-24 21:48:54,284|[WARNING]|10216|9096|dlt|client.py|_normalize_identifier:245|Due to normalization dataset name got changed from 'MySql_pipelines_dataset_20250925044853' to 'my_sql_pipelines_dataset_20250925044853 which will be used to create db schemas or folders. `dataset_name` field in the pipeline instance will not be changed. We suggest that you use dataset names that do not need to be normalized or disable dataset name normalization via `enable_dataset_name_normalization` on destination configuration.
2025-09-24 21:48:54,284|[WARNING]|10216|9096|dlt|client.py|_normalize_identifier:245|Due to normalization dataset name got changed from 'MySql_pipelines_dataset_20250925044853_staging' to 'my_sql_pipelines_dataset_20250925044853_staging which will be used to create db schemas or folders. `dataset_name` field in the pipeline instance will not be changed. We suggest that you use dataset names that do not need to be normalized or disable dataset name normalization via `enable_da

Pipeline MySql_pipelines load step completed in 0.46 seconds
1 load package(s) were loaded to destination duckdb and into dataset my_sql_pipelines_dataset_20250925044853
The duckdb destination used duckdb:///h:\Data Engineering\Projects\DE Projects\ETL_DLT_pipeline\MySql_pipelines.duckdb location to store data
Load package 1758775733.7837393 is LOADED and contains no failed jobs


In [33]:
#Slowly changing dimensions: (SCD 2 example)
data = [
    {"name": "Vincent Crabbe", "designation": "student", "date_started": "1991-09-01T09:00:00Z"},
    {"name": "Gregory Goyle", "designation": "student", "date_started": "1991-09-01T09:00:00Z"},
    {"name": "Draco Malfoy", "designation": "student", "date_started": "1991-09-01T09:00:00Z"}
]

data_updated = [
    {"name": "Vincent Crabbe", "designation": "student", "date_started": "1991-09-01T09:00:00Z"},
    {"name": "Gregory Goyle", "designation": "student", "date_started": "1991-09-01T09:00:00Z"},
    {"name": "Draco Malfoy", "designation": "expelled", "date_started": "1991-09-01T09:00:00Z"}
]

In [34]:
#Run pipeline with initial dataset:
import dlt

pipeline = dlt.pipeline(
    pipeline_name="hogwarts_pipeline",
    destination="duckdb",
    dataset_name="hogwarts",
)

load_info = pipeline.run(
    data,
    table_name="creatures",
    write_disposition={
        "disposition": "merge", # <--- specifies that existing data should be merged rather than replaced
        "strategy": "scd2" # <--- enables SCD2 tracking, which keeps historical records of changes
    }
)
print(load_info)

pipeline.dataset().creatures.df()

Pipeline hogwarts_pipeline load step completed in 0.44 seconds
1 load package(s) were loaded to destination duckdb and into dataset hogwarts
The duckdb destination used duckdb:///h:\Data Engineering\Projects\DE Projects\ETL_DLT_pipeline\hogwarts_pipeline.duckdb location to store data
Load package 1758784144.384754 is LOADED and contains no failed jobs


,_dlt_valid_from,_dlt_valid_to,name,designation,date_started,_dlt_load_id,_dlt_id
0,2025-09-25 07:05:37.934117+00:00,NaT,Vincent Crabbe,student,1991-09-01 09:00:00+00:00,1758783937.9341166,P94fUt8jUkzx6A
1,2025-09-25 07:05:37.934117+00:00,NaT,Gregory Goyle,student,1991-09-01 09:00:00+00:00,1758783937.9341166,Wv4kISjkizZwkQ
2,2025-09-25 07:05:37.934117+00:00,2025-09-25 07:06:00.934666+00:00,Draco Malfoy,student,1991-09-01 09:00:00+00:00,1758783937.9341166,r7u0vNojh+gvEQ
3,2025-09-25 07:06:00.934666+00:00,2025-09-25 07:08:46.801204+00:00,Draco Malfoy,expelled,1991-09-01 09:00:00+00:00,1758783960.934666,KCWs+o6jOW/aLQ
4,2025-09-25 07:08:46.801204+00:00,2025-09-25 07:09:04.384754+00:00,Maximo Malfoy,expelled,1991-09-01 09:00:00+00:00,1758784126.801204,HcX8XVCthXfi5Q
5,2025-09-25 07:09:04.384754+00:00,NaT,Draco Malfoy,student,1991-09-01 09:00:00+00:00,1758784144.384754,r7u0vNojh+gvEQ


In [35]:
#Run pipeline with updated dataset:
load_info = pipeline.run(
    data_updated,
    table_name="creatures",
    write_disposition={
        "disposition": "merge", # <--- specifies that existing data should be merged rather than replaced
        "strategy": "scd2" # <--- enables SCD2 tracking, which keeps historical records of changes
    }
)
print(load_info)

pipeline.dataset().creatures.df()

Pipeline hogwarts_pipeline load step completed in 0.39 seconds
1 load package(s) were loaded to destination duckdb and into dataset hogwarts
The duckdb destination used duckdb:///h:\Data Engineering\Projects\DE Projects\ETL_DLT_pipeline\hogwarts_pipeline.duckdb location to store data
Load package 1758784151.6306615 is LOADED and contains no failed jobs


,_dlt_valid_from,_dlt_valid_to,name,designation,date_started,_dlt_load_id,_dlt_id
0,2025-09-25 07:05:37.934117+00:00,NaT,Vincent Crabbe,student,1991-09-01 09:00:00+00:00,1758783937.9341166,P94fUt8jUkzx6A
1,2025-09-25 07:05:37.934117+00:00,NaT,Gregory Goyle,student,1991-09-01 09:00:00+00:00,1758783937.9341166,Wv4kISjkizZwkQ
2,2025-09-25 07:05:37.934117+00:00,2025-09-25 07:06:00.934666+00:00,Draco Malfoy,student,1991-09-01 09:00:00+00:00,1758783937.9341166,r7u0vNojh+gvEQ
3,2025-09-25 07:06:00.934666+00:00,2025-09-25 07:08:46.801204+00:00,Draco Malfoy,expelled,1991-09-01 09:00:00+00:00,1758783960.934666,KCWs+o6jOW/aLQ
4,2025-09-25 07:08:46.801204+00:00,2025-09-25 07:09:04.384754+00:00,Maximo Malfoy,expelled,1991-09-01 09:00:00+00:00,1758784126.801204,HcX8XVCthXfi5Q
5,2025-09-25 07:09:04.384754+00:00,2025-09-25 07:09:11.630661+00:00,Draco Malfoy,student,1991-09-01 09:00:00+00:00,1758784144.384754,r7u0vNojh+gvEQ
6,2025-09-25 07:09:11.630661+00:00,NaT,Draco Malfoy,expelled,1991-09-01 09:00:00+00:00,1758784151.6306615,KCWs+o6jOW/aLQ
